In [1]:
import pandas as pd
from stackapi import StackAPI
from pprint import pprint
from bs4 import BeautifulSoup
import time
import requests
import html5lib
import bs4
import pickle
from tqdm import tqdm
from joblib import Parallel, delayed

In [2]:
df = pd.read_csv('Dataframe_AskUbuntu.csv')

C:\Users\madhu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
len(df)

74190

In [4]:
bash_file_codes=pickle.load(open('posts1.txt', 'rb'))
failed=[]

In [6]:
for i in tqdm(range(0, len(df))):
    if(i%5000 == 0):
        time.sleep(600)
    try:
        answers={}
        codes=[]
        i_d = df.iloc[i, 7]
        i_d = int(i_d)
        url = 'https://askubuntu.com/questions/'+str(i_d)
        r = requests.get(url) 
        soup = BeautifulSoup(r.content, 'html.parser')
        j=0

        q = soup.find('div', attrs={'class':"s-prose js-post-body", 'itemprop':"text"})



        for a in soup.findAll('div', attrs={'class':'answer accepted-answer'}):
            atext = a.text
            aid = a['id']
            ans = {}
            codes=[]
            c = a.find('div', attrs={'class':"js-vote-count grid--cell fc-black-500 fs-title grid fd-column ai-center", 'itemprop':"upvoteCount"})

            for b in a.findAll('div', attrs={'class':'s-prose js-post-body'}):
                for code in b.findAll('code'):
                    codes.append(code.text)

            ans['Answer Id'] = aid
            ans['Answer Text'] = atext

            if(c.text):
                ans['Votes'] = c.text
            else:
                ans['Votes'] = 0

            ans['Codes'] = codes

            answers['Accepted Answer'] = ans



        for a in soup.findAll('div', attrs={'class':'answer'}):
            atext = a.text
            aid = a['id']
            ans = {}
            codes=[]
            j=j+1
            c = a.find('div', attrs={'class':"js-vote-count grid--cell fc-black-500 fs-title grid fd-column ai-center", 'itemprop':"upvoteCount"})

            for b in a.findAll('div', attrs={'class':'s-prose js-post-body'}):
                for code in b.findAll('code'):
                    codes.append(code.text)

            ans['Answer Id'] = aid
            ans['Answer Text'] = atext

            if(c.text):
                ans['Votes'] = c.text
            else:
                ans['Votes'] = 0

            ans['Codes'] = codes
            answers['Answer'+str(j)] = ans

            if(j>=2):
                break

        bash_file_codes.append({'Id':i_d, 'Question Body': q.text, 'Answers':answers})
        if(i%2==0):
            pickle.dump(bash_file_codes, open('posts2.txt', 'wb'))
        else:
            pickle.dump(bash_file_codes, open('posts1.txt', 'wb'))
            

    except:
        failed.append(i)
        pickle.dump(failed, open('failed.txt', 'wb'))

100%|██████████| 3190/3190 [1:43:35<00:00,  1.95s/it]
